In [1]:
import string
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize,sent_tokenize
from nltk.tokenize import TweetTokenizer
with open('C:\\Python\\train_90.txt',encoding="utf8",mode="r") as InpFile:
    data=InpFile.readlines()

from nltk.corpus import stopwords
stp_wrd=set(stopwords.words('english'))

In [2]:
import pandas as pd
df = pd.read_csv('C:\\Python\\train_90.txt', header=None,names= ["ItemID", "Sentiment", "SentimentSource", "SentimentText"],
                 skiprows=1)

In [3]:
l1=df['SentimentText']
ytrain=df['Sentiment']

Data Cleaning------------->>>>>>>>>>>>>

In [4]:
def SentTokenize(data):
    tweettknzr = TweetTokenizer(strip_handles=True)
    l2=list()
    for i in list(range(0,len(data))):
        l2.append(tweettknzr.tokenize(''.join(data[i]))) # Tokenized the words at row level
    return(l2)

def CleanTokenize(data):
    TokenizeClean=list()
    for i in list(range(0,len(data))):
        temp=list()
        for j in list(range(len(data[i]))):
            if(data[i][j].isalpha() or data[i][j].isnumeric()): # Removed punctuations and special chars by keeping only alphanumerics
                temp.append(data[i][j].lower())              # Converted words to lower case
        TokenizeClean.append(temp)
    return(TokenizeClean)

def RemoveStopwords(data):
    l3=list()
    for i in list(range(0,len(data))):
        temp=list()
        for j in list(range(len(data[i]))):
            if(data[i][j] not in stp_wrd): # Removed Stopwords from the nltk corpus
                temp.append(data[i][j])
        l3.append(temp)
    return(l3)

def TweetStemming(data):
    from nltk.stem import PorterStemmer
    ps=PorterStemmer()
    l3=list()
    for i in list(range(0,len(data))):
        temp=list()
        for j in list(range(len(data[i]))):
            temp.append(ps.stem(data[i][j]))
        l3.append(temp)
    return(l3)

In [5]:
xtraintokenize=CleanTokenize(SentTokenize(l1))
StemmedTweets=TweetStemming(RemoveStopwords(xtraintokenize))

TF/IDF Calculations------->>>>>>>>

In [6]:
#----Converting each tweet to String
from sklearn.feature_extraction.text import TfidfVectorizer  
import pickle
    
TweetForIDF=list()
str=""
for i in list(range(0,len(StemmedTweets))):
    temp=list()
    str=""
    str=(" ".join(StemmedTweets[i]))
    TweetForIDF.append(str)
    
## --- min_df=10 --The tweet words must occur in atleast 10 tweets in order to consider it as an important word for classification
## --- max_df=0.7 -- If a word occurs in more than 70% of documents, that means it is not important for classification

tfidfconverter = TfidfVectorizer(min_df=10, max_df=0.7, stop_words=stopwords.words('english'))  
xtrain = tfidfconverter.fit_transform(TweetForIDF).toarray()
pickle.dump(tfidfconverter, open('tfidfconverter.l', 'wb')) ##---- Storing the training corpus

##---- Processing Test Data

In [7]:
testdf = pd.read_csv('C:\\Python\\test_10.txt', header=None,names= ["ItemID", "Sentiment", "SentimentSource", "SentimentText"],
                 skiprows=1) ##---- Input Test Data

testtweet=testdf['SentimentText']
ytest10=testdf['Sentiment']

xtesttokenize=CleanTokenize(SentTokenize(testtweet))
testtweetclean=TweetStemming(RemoveStopwords(xtesttokenize)) ###----Clean-Tokenized/Stemmed test data

#----Converting each tweet to String
from sklearn.feature_extraction.text import TfidfVectorizer  
import pickle
    
TweetForIDF=list()
str=""
for i in list(range(0,len(testtweetclean))):
    temp=list()
    str=""
    str=(" ".join(testtweetclean[i]))
    TweetForIDF.append(str)
    
## --- min_df=10 --The tweet words must occur in atleast 10 tweets in order to consider it as an important word for classification
## --- max_df=0.7 -- If a word occurs in more than 70% of documents, that means it is not important for classification

tfidfconverter = pickle.load(open('tfidfconverter.l', 'rb')) ##---- Retrieving the corpus of training words
xtest = tfidfconverter.transform(TweetForIDF).toarray()      ## --- TF/IDF of test data based on training corpus


In [8]:
print(xtrain.shape)
print(xtest.shape)

(89989, 4653)
(10000, 4653)


##----- Functions for Logistic Regression Model------>>>>>>>>

In [9]:
##---- Sigmoid Function------

def sigmoidfunction(x):
    return (1/(1+np.exp(-x)))

##--- Computing Theta.X
def theta_x(theta,x):
    return (np.dot(x,theta))

##------ Probability function -----
def retprob(theta,x):
    return( sigmoidfunction(theta_x(theta,x)))

##------ Logistic Loss Function
def lossfunction(h,y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h))

##------ Calculating gradient for Logistic loss function
def gradientcalc(x,y,h,theta,learningrate,numiters):
    for i in range(0,numiters):
        grad=np.dot(x.T,(h-y)) / y.shape[0]
        thetanew=theta - learningrate*grad
        theta=thetanew
    return(thetanew)
    
##------ Predicting test/validation label
def predictions(x,theta):
    thet = theta[:, np.newaxis]
    return retprob(thet, x)

##--- Computing accuracy
def accuracycalc(xtest,ytest,theta):
    predictedclass=((predictions(xtest,theta)>=0.5) .astype(int))
    predictedclass=predictedclass.flatten()
    
    accuracy=np.mean(predictedclass==ytest)
    
    return (accuracy)

Cross Validation--------->>>>>>>>>>

In [10]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)         ###---- 10-fold cross validation
accuracy=list()
theta = np.zeros(xtrain.shape[1])
thetabest=theta.shape
bestaccuracy=0
numiters=200

for train, test in kf.split(xtrain):
    
    theta = np.zeros(xtrain.shape[1])
    
    xtraining=xtrain[train]
    ytraining=ytrain[train]
    xvalidate=xtrain[test]
    yvalidate=ytrain[test]
    
    h=retprob(theta,xtraining)
    print('Pass Started---->')
    theta=gradientcalc(xtraining,ytraining,h,theta,0.1,numiters)
    print('Pass Finished---->')
    #predictedval=predictions(xtraining,theta)
    
    accuracyval=accuracycalc(xvalidate,yvalidate,theta)
    
    #---- Storing the accuracy and theta values based on best performance on validation set
    
    if(accuracyval>bestaccuracy):
        thetabest=theta
        bestaccuracy=accuracyval
    
    accuracy.append(accuracyval)

Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->
Pass Started---->
Pass Finished---->


In [11]:
##--- Accuracy on test data

acc=accuracycalc(xtest,ytest10,thetabest)
acc

0.7096

In [12]:
##--- Confusion Matrix -- Test Data

testpredict=((predictions(xtest,thetabest)>0.5).astype(int)).flatten()
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest10,testpredict)


array([[1897, 2109],
       [ 847, 5147]], dtype=int64)

In [13]:
##-- Recall / Precision -- Test Data

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print("Recall->",recall_score(ytest10,testpredict))
print("Precision->",precision_score(ytest10,testpredict))

Recall-> 0.858692025358692
Precision-> 0.7093439911797134
